# Wrangling and Analyzing Data - We Rate Dogs

## Lets start by gathering the modules we will need for this project and loading environment variables

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import tweepy
import os
from dotenv import load_dotenv
load_dotenv()

True

##  Data Gathering

### We are going to need three sets of data for this project. Let's gather those now

#### Starting with import the twitter archive csv

In [2]:
archive = pd.read_csv("./twitter-archive-enhanced.csv")
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

#### The next is a tsv hosted on a third party site. The project requires us to use the requests library to retrieve this file, however we can also just load it through pandas if we didn't want to use local hard drive space.

requests example

In [3]:
prediction_file = requests.get("https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv")
with open("./predictions_file.tsv","wb") as f:
  f.write(prediction_file.content)
predictions = pd.read_csv("./predictions_file.tsv", delimiter="\t")
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


Pandas example

In [4]:
predictions2 = pd.read_csv("https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv", delimiter="\t")
predictions2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [5]:
(predictions != predictions2).sum()

tweet_id    0
jpg_url     0
img_num     0
p1          0
p1_conf     0
p1_dog      0
p2          0
p2_conf     0
p2_dog      0
p3          0
p3_conf     0
p3_dog      0
dtype: int64

As these files are identical, it doesn't matter which one we use.

#### Finally we have the Twitter Data gathered through the use of the Twitter API and Tweepy


##### Please note thta the following code was operational before Twitter moved it's data behind a paywall, as evidenced by the json files made from it, however it currently returns `403 Forbidden` errors and as such is commented out

In [6]:
# client = tweepy.Client(os.environ["TWITTER_BEARER"], wait_on_rate_limit=True)

# def get_data_by_tweet_ids(tweet_ids, file_obj):
#   try:
#     data = client.get_tweets(tweet_ids, tweet_fields=["public_metrics"]).data
#     for tweet in data:
#       try:
#         write_data = {"id":tweet.data["id"], **tweet.data["public_metrics"]}
#         file_obj.write(f'{write_data},\n'.replace("'",'"'))
#       except Exception as e:
#         print(f"{tweet}:{e}")
#   except Exception as e:
#     print(e)

In [7]:
# full_list = list(archive["tweet_id"])
# query_lists = [full_list[i:i+100] for i in range(0,len(full_list),100)]
# with open("./tweet_json.txt","w") as f:
#   for l in query_lists:
#     get_data_by_tweet_ids(l,f)

##### While the project specifies reading this line by line into a dataframe, we can also convert it into json and read it using `pd.read_json`

line by line example

In [8]:
# with open("./tweet_json.txt","r") as f:
#   data_list = [ json.loads(line[:-2]) for line in f.readlines()]
# twitter_data = pd.DataFrame(data_list)
# twitter_data.info()

In [9]:
# with open("./tweet_json.txt","r") as old_f:
#   lines = old_f.readlines()
#   lines[-1] = lines[-1][:-2]
#   with open("./tweet_json.json","w") as new_f:
#     new_f.writelines(["[\n", *lines, "\n]"])
twitter_df = pd.read_json("./tweet_json.json", orient="records")
twitter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2326 entries, 0 to 2325
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   id                2326 non-null   int64
 1   retweet_count     2326 non-null   int64
 2   reply_count       2326 non-null   int64
 3   like_count        2326 non-null   int64
 4   quote_count       2326 non-null   int64
 5   impression_count  2326 non-null   int64
dtypes: int64(6)
memory usage: 109.2 KB


In [10]:
# (twitter_data != twitter_df).sum()

The only real difference between the two is that the first method keeps the ids as strings, where the json converts it to int64s. considering the other dataframes store them as int64 we will continue using the second dataframe.

## Assessing Data

### Lets start by analyzing the original archive provided

#### Visual Analysis (to be continued in excel)

In [11]:
archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None


#### Programmatic Analysis

In [12]:
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

##### First let's look for duplicates

In [13]:
archive.duplicated(subset=["tweet_id"]).sum()

0

##### None of those, now looking for nulls

In [14]:
archive.isna().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

#### Archive Issues
##### Cleanliness
- The `source` column is full of anchor tags that are unnecessary
- Several of the `name` values are simply 'a' or 'None' which should be replaced with `na`
- Some of the records do not have the URL of the tweet
- We don't want any of the retweeted rows in the dataset
- `in_reply_to_status_id` has a great deal of null values
- `in_reply_to_user_id` has a great deal of null values
- `timestamp` is a string, and should be a datetime
- classification columns (doggo, floofer, pupper, puppo) are strings and should be booleans
##### Tidiness
- There are multiple columns for the classification of the dog (doggo, floofer, pupper, puppo), and should be reduced to a single column

### Now we should handle the predicitons file

#### Visual (once again continued externally)

In [15]:
predictions

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


#### Programatic

In [16]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [17]:
predictions.duplicated().sum()

0

In [18]:
predictions.isna().sum()

tweet_id    0
jpg_url     0
img_num     0
p1          0
p1_conf     0
p1_dog      0
p2          0
p2_conf     0
p2_dog      0
p3          0
p3_conf     0
p3_dog      0
dtype: int64

In [19]:
predictions.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [20]:
sum([predictions["p1_conf"], predictions["p2_conf"], predictions["p3_conf"]]).describe()

count    2075.000000
mean        0.789461
std         0.208969
min         0.126448
25%         0.657151
50%         0.855279
75%         0.971044
max         1.000000
dtype: float64

#### Predicitons Issues
##### Cleanliness
- Some of the predictions don't have a dog value in them

### Finally let's look at our twitter data

#### Visual

In [21]:
twitter_df

,id,retweet_count,reply_count,like_count,quote_count,impression_count
0,892420643555336192,6848,135,32800,219,0
1,892177421306343424,5159,162,28345,274,0
2,891815181378084864,3405,108,21289,145,0
3,891689557279858688,7056,139,35741,286,0
4,891327558926688256,7568,192,34180,296,0
...,...,...,...,...,...,...
2321,666049248165822464,36,9,85,0,0
2322,666044226329800704,113,1,242,2,0
2323,666033412701032448,35,1,97,0,0
2324,666029285002620928,39,0,109,0,0


#### Programmatic

In [22]:
twitter_df.duplicated().sum()

0

In [23]:
twitter_df.describe()

,id,retweet_count,reply_count,like_count,quote_count,impression_count
count,2.326000e+03,2326.000000,2326.000000,2326.000000,2326.000000,2326.0
mean,7.417346e+17,2414.525795,41.263543,6844.829321,71.599312,0.0
std,6.816442e+16,4087.231292,69.257771,10634.416751,220.423821,0.0
min,6.660209e+17,0.000000,0.000000,0.000000,0.000000,0.0
25%,6.780814e+17,484.500000,9.000000,1190.250000,0.000000,0.0
50%,7.178159e+17,1123.500000,23.000000,2963.500000,1.000000,0.0
75%,7.986402e+17,2793.750000,47.000000,8370.250000,69.750000,0.0
max,8.924206e+17,69089.000000,1100.000000,141185.000000,4352.000000,0.0


#### Twitter Issues
##### Tidiness
- `impression_count` is an unnecessary column with all 0's

## Cleaning and storing

### Import `sqlalchemy` and create our connection to work with a sqlite3 db

In [24]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///we_rate_dogs.sqlite3', echo=False)
conn = engine.connect()

### Cleaning Archive table

#### Let's start with the cleanliness issues

In [25]:
def remove_anchors(s: str) -> str:
    return s.split('rel="nofollow">')[-1].split("</a>")[0]

archive["source"] = archive["source"].apply(remove_anchors)
assert all(map(lambda x: "<a" not in x ,archive["source"].unique()))

In [26]:
name_filter = archive["name"].isin(["a", "None"])
archive.loc[name_filter, "name"] = None
assert archive["name"].isna().sum() > 0

In [27]:
def build_url(s:int) -> str:
    return f"https://twitter.com/dog_rates/status/{s}"
archive_filter = archive["expanded_urls"].isna()
archive.loc[archive_filter, "expanded_urls"] = archive.loc[archive_filter, "tweet_id"].apply(build_url)
assert archive["expanded_urls"].isna().sum() == 0

In [28]:
dropped_columns = ["retweeted_status_id","retweeted_status_user_id","retweeted_status_timestamp"]
dropped_rows = ~archive[dropped_columns].isna().any(axis=1)
archive = archive.drop(archive[dropped_rows].index)
assert (~archive[dropped_columns].isna().any(axis=1)).sum() == 0
archive = archive.drop(dropped_columns, axis=1)
assert all(map(lambda x: x not in archive.columns, dropped_columns))

In [29]:
## We will just drop in_reply_to columns as the data isn't that helpful to us
dropped_columns = [col for col in archive.columns if "in_reply_to" in col]
assert len(dropped_columns) == 2
archive = archive.drop(dropped_columns, axis=1)
dropped_columns = [col for col in archive.columns if "in_reply_to" in col]
assert all(map(lambda x: x not in archive.columns, dropped_columns))

In [30]:
archive["timestamp"] = pd.to_datetime(archive["timestamp"])
assert isinstance(archive.loc[0,"timestamp"], pd.Timestamp)

In [31]:
def str_to_bool(s:str) -> bool:
    return s != "None"
cat_columns = archive.columns[-4:]
for col in cat_columns:
    archive[col] = archive[col].apply(str_to_bool)

archive[cat_columns].sum()

doggo       87
floofer     10
pupper     234
puppo       25
dtype: int64

#### Now for Tidiness